# Imports

In [76]:
import sys
print(sys.path)

['/Users/thibaultdesfontaines/git-kaggling-chaos/kaggle-ieee-fraud-detection', '/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python37.zip', '/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7', '/usr/local/Cellar/python/3.7.6_1/Frameworks/Python.framework/Versions/3.7/lib/python3.7/lib-dynload', '', '/usr/local/lib/python3.7/site-packages', '/usr/local/lib/python3.7/site-packages/geos', '/usr/local/Cellar/protobuf/3.11.3/libexec/lib/python3.7/site-packages', '/usr/local/lib/python3.7/site-packages/IPython/extensions', '/Users/thibaultdesfontaines/.ipython']


In [1]:
import os
import pandas as pd

In [2]:
import numpy as np, pandas as pd, os, gc
from sklearn.model_selection import GroupKFold
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns

In [26]:
DATA_PATH = "/Users/thibaultdesfontaines/data/ieee-fraud-detection/"
os.listdir(DATA_PATH)

['test_transaction.csv',
 '.DS_Store',
 'train_identity.csv',
 'test_identity.csv',
 'sample_submission.csv',
 'train_transaction.csv']

In [3]:
BUILD95 = True
BUILD96 = True

Specify columns with strings:

In [4]:
str_type = ['ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain','M1', 'M2', 'M3', 'M4','M5',
            'M6', 'M7', 'M8', 'M9', 'id_12', 'id_15', 'id_16', 'id_23', 'id_27', 'id_28', 'id_29', 'id_30', 
            'id_31', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo']
str_type += ['id-12', 'id-15', 'id-16', 'id-23', 'id-27', 'id-28', 'id-29', 'id-30', 
            'id-31', 'id-33', 'id-34', 'id-35', 'id-36', 'id-37', 'id-38']

Isolate the 53 first columns:

In [5]:
cols = ['TransactionID', 'TransactionDT', 'TransactionAmt',
       'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6',
       'addr1', 'addr2', 'dist1', 'dist2', 'P_emaildomain', 'R_emaildomain',
       'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11',
       'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8',
       'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'M1', 'M2', 'M3', 'M4',
       'M5', 'M6', 'M7', 'M8', 'M9']

In [6]:
v =  [1, 3, 4, 6, 8, 11]
v += [13, 14, 17, 20, 23, 26, 27, 30]
v += [36, 37, 40, 41, 44, 47, 48]
v += [54, 56, 59, 62, 65, 67, 68, 70]
v += [76, 78, 80, 82, 86, 88, 89, 91]
v += [107, 108, 111, 115, 117, 120, 121, 123]
v += [124, 127, 129, 130, 136]
v += [138, 139, 142, 147, 156, 162]
v += [165, 160, 166]
v += [178, 176, 173, 182]
v += [187, 203, 205, 207, 215]
v += [169, 171, 175, 180, 185, 188, 198, 210, 209]
v += [218, 223, 224, 226, 228, 229, 235]
v += [240, 258, 257, 253, 252, 260, 261]
v += [264, 266, 267, 274, 277]
v += [220, 221, 234, 238, 250, 271]
v += [294, 284, 285, 286, 291, 297]
v += [303, 305, 307, 309, 310, 320]
v += [281, 283, 289, 296, 301, 314]

In [7]:
cols += ['V'+str(x) for x in v]

Specify dtypes:

In [15]:
dtypes = {}
for c in cols+['id_0'+str(x) for x in range(1,10)]+['id_'+str(x) for x in range(10,34)]+\
    ['id-0'+str(x) for x in range(1,10)]+['id-'+str(x) for x in range(10,34)]:
        dtypes[c] = 'float32'
for c in str_type: dtypes[c] = 'category'

In [27]:
%%time
X_train = pd.read_csv(DATA_PATH+'train_transaction.csv',
                      index_col='TransactionID', dtype=dtypes, usecols=cols+['isFraud'])

CPU times: user 9.95 s, sys: 475 ms, total: 10.4 s
Wall time: 10.4 s


In [28]:
%%time 
train_id = pd.read_csv(DATA_PATH+'train_identity.csv',
                       index_col='TransactionID', dtype=dtypes)

CPU times: user 512 ms, sys: 29.1 ms, total: 541 ms
Wall time: 540 ms


In [29]:
%%time 
X_test = pd.read_csv(DATA_PATH+'test_transaction.csv',
                     index_col='TransactionID', dtype=dtypes, usecols=cols)

CPU times: user 8.62 s, sys: 386 ms, total: 9 s
Wall time: 9.01 s


In [30]:
%%time
test_id = pd.read_csv(DATA_PATH+'test_identity.csv', 
                      index_col='TransactionID', dtype=dtypes)

CPU times: user 528 ms, sys: 27.9 ms, total: 556 ms
Wall time: 555 ms


In [31]:
fix = {o: n for o, n in zip(test_id.columns, train_id.columns)}

In [33]:
test_id.rename(columns=fix, inplace=True)

In [34]:
X_test = X_test.merge(test_id, how='left', left_index=True, right_index=True)

In [35]:
y_train = X_train['isFraud'].copy()

In [36]:
del train_id, test_id, X_train['isFraud']
x = gc.collect()

In [38]:
print('Train shape', X_train.shape, 'test shape', X_test.shape)

Train shape (590540, 173) test shape (506691, 213)


Normalize the D columns:

In [41]:
for i in range(1,16):
    if i in [1,2,3,5,9]: continue
    X_train['D'+str(i)] =  X_train['D'+str(i)] - X_train.TransactionDT/np.float32(24*60*60)
    X_test['D'+str(i)] = X_test['D'+str(i)] - X_test.TransactionDT/np.float32(24*60*60) 

In [52]:
%%time
# LABEL ENCODE AND MEMORY REDUCE
for i,f in enumerate(X_train.columns):
    # FACTORIZE CATEGORICAL VARIABLES
    if (np.str(X_train[f].dtype)=='category')|(X_train[f].dtype=='object'): 
        df_comb = pd.concat([X_train[f],X_test[f]],axis=0)
        df_comb,_ = df_comb.factorize(sort=True)
        if df_comb.max()>32000: print(f,'needs int32')
        X_train[f] = df_comb[:len(X_train)].astype('int16')
        X_test[f] = df_comb[len(X_train):].astype('int16')
    # SHIFT ALL NUMERICS POSITIVE. SET NAN to -1
    elif f not in ['TransactionAmt','TransactionDT']:
        mn = np.min((X_train[f].min(),X_test[f].min()))
        X_train[f] -= np.float32(mn)
        X_test[f] -= np.float32(mn)
        X_train[f].fillna(-1,inplace=True)
        X_test[f].fillna(-1,inplace=True)

CPU times: user 1.85 s, sys: 91.1 ms, total: 1.95 s
Wall time: 1.95 s


Encoding functions:

In [47]:
def encode_FE(df1, df2, cols):
    for col in cols:
        df = pd.concat([df1[col],df2[col]])
        vc = df.value_counts(dropna=True, normalize=True).to_dict()
        vc[-1] = -1
        nm = col+'_FE'
        df1[nm] = df1[col].map(vc)
        df1[nm] = df1[nm].astype('float32')
        df2[nm] = df2[col].map(vc)
        df2[nm] = df2[nm].astype('float32')
        print(nm,', ',end='')

In [48]:
def encode_LE(col,train=X_train,test=X_test,verbose=True):
    df_comb = pd.concat([train[col],test[col]],axis=0)
    df_comb,_ = df_comb.factorize(sort=True)
    nm = col
    if df_comb.max()>32000: 
        train[nm] = df_comb[:len(train)].astype('int32')
        test[nm] = df_comb[len(train):].astype('int32')
    else:
        train[nm] = df_comb[:len(train)].astype('int16')
        test[nm] = df_comb[len(train):].astype('int16')
    del df_comb; x=gc.collect()
    if verbose: print(nm,', ',end='')

In [49]:
def encode_AG(main_columns, uids, aggregations=['mean'], train_df=X_train, test_df=X_test, 
              fillna=True, usena=False):
    # AGGREGATION OF MAIN WITH UID FOR GIVEN STATISTICS
    for main_column in main_columns:  
        for col in uids:
            for agg_type in aggregations:
                new_col_name = main_column+'_'+col+'_'+agg_type
                temp_df = pd.concat([train_df[[col, main_column]], test_df[[col,main_column]]])
                if usena: temp_df.loc[temp_df[main_column]==-1,main_column] = np.nan
                temp_df = temp_df.groupby([col])[main_column].agg([agg_type]).reset_index().rename(
                                                        columns={agg_type: new_col_name})

                temp_df.index = list(temp_df[col])
                temp_df = temp_df[new_col_name].to_dict()   

                train_df[new_col_name] = train_df[col].map(temp_df).astype('float32')
                test_df[new_col_name]  = test_df[col].map(temp_df).astype('float32')
                
                if fillna:
                    train_df[new_col_name].fillna(-1,inplace=True)
                    test_df[new_col_name].fillna(-1,inplace=True)
                
                print("'"+new_col_name+"'",', ',end='')

In [50]:
def encode_CB(col1,col2,df1=X_train,df2=X_test):
    nm = col1+'_'+col2
    df1[nm] = df1[col1].astype(str)+'_'+df1[col2].astype(str)
    df2[nm] = df2[col1].astype(str)+'_'+df2[col2].astype(str) 
    encode_LE(nm,verbose=False)
    print(nm,', ',end='')

In [51]:
def encode_AG2(main_columns, uids, train_df=X_train, test_df=X_test):
    for main_column in main_columns:  
        for col in uids:
            comb = pd.concat([train_df[[col]+[main_column]],test_df[[col]+[main_column]]],axis=0)
            mp = comb.groupby(col)[main_column].agg(['nunique'])['nunique'].to_dict()
            train_df[col+'_'+main_column+'_ct'] = train_df[col].map(mp).astype('float32')
            test_df[col+'_'+main_column+'_ct'] = test_df[col].map(mp).astype('float32')
            print(col+'_'+main_column+'_ct, ',end='')

Feature engineering

In [53]:
%%time
X_train['cents'] = (X_train['TransactionAmt'] - np.floor(X_train['TransactionAmt'])).astype('float32')
X_test['cents'] = (X_test['TransactionAmt'] - np.floor(X_test['TransactionAmt'])).astype('float32')
print('cents, ', end='')

cents, CPU times: user 7.73 ms, sys: 2.06 ms, total: 9.8 ms
Wall time: 7.72 ms


In [54]:
%%time 
encode_FE(X_train,X_test,['addr1','card1','card2','card3','P_emaildomain'])

addr1_FE , card1_FE , card2_FE , card3_FE , P_emaildomain_FE , CPU times: user 248 ms, sys: 30.3 ms, total: 278 ms
Wall time: 275 ms


In [55]:
%%time
encode_CB('card1','addr1')

card1_addr1 , CPU times: user 1.32 s, sys: 95.5 ms, total: 1.42 s
Wall time: 1.42 s


In [56]:
%%time 
encode_CB('card1_addr1','P_emaildomain')

card1_addr1_P_emaildomain , CPU times: user 1.87 s, sys: 90.6 ms, total: 1.96 s
Wall time: 1.97 s


In [57]:
%%time
encode_FE(X_train,X_test,['card1_addr1','card1_addr1_P_emaildomain'])

card1_addr1_FE , card1_addr1_P_emaildomain_FE , CPU times: user 706 ms, sys: 42.5 ms, total: 749 ms
Wall time: 749 ms


In [58]:
%%time
encode_AG(['TransactionAmt','D9','D11'],
          ['card1','card1_addr1','card1_addr1_P_emaildomain'],
          ['mean','std'], usena=True)

'TransactionAmt_card1_mean' , 'TransactionAmt_card1_std' , 'TransactionAmt_card1_addr1_mean' , 'TransactionAmt_card1_addr1_std' , 'TransactionAmt_card1_addr1_P_emaildomain_mean' , 'TransactionAmt_card1_addr1_P_emaildomain_std' , 'D9_card1_mean' , 'D9_card1_std' , 'D9_card1_addr1_mean' , 'D9_card1_addr1_std' , 'D9_card1_addr1_P_emaildomain_mean' , 'D9_card1_addr1_P_emaildomain_std' , 'D11_card1_mean' , 'D11_card1_std' , 'D11_card1_addr1_mean' , 'D11_card1_addr1_std' , 'D11_card1_addr1_P_emaildomain_mean' , 'D11_card1_addr1_P_emaildomain_std' , CPU times: user 14.5 s, sys: 7.59 s, total: 22.1 s
Wall time: 22.2 s


Feature Selection - Time Consistency

In [69]:
cols = list( X_train.columns )

In [70]:
cols.remove('TransactionDT')

for c in ['D6','D7','D8','D9','D12','D13','D14']:
    try:
        cols.remove(c)
    except:
        print("{} not in cols".format(c))
        
for c in ['C3','M5','id_08','id_33']:
    try:
        cols.remove(c)
    except:
        print("{} not in cols".format(c))
        
for c in ['card4','id_07','id_14','id_21','id_30','id_32','id_34']:
    try:
        cols.remove(c)
    except:
        print("{} not in cols".format(c))
        
for c in ['id_'+str(x) for x in range(22,28)]:
    try:
        cols.remove(c)
    except:
        print("{} not in cols".format(c))

id_08 not in cols
id_33 not in cols
id_07 not in cols
id_14 not in cols
id_21 not in cols
id_30 not in cols
id_32 not in cols
id_34 not in cols
id_22 not in cols
id_23 not in cols
id_24 not in cols
id_25 not in cols
id_26 not in cols
id_27 not in cols


In [71]:
np.array(cols)

array(['TransactionAmt', 'ProductCD', 'card1', 'card2', 'card3', 'card5',
       'card6', 'addr1', 'addr2', 'dist1', 'dist2', 'P_emaildomain',
       'R_emaildomain', 'C1', 'C2', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9',
       'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5',
       'D10', 'D11', 'D15', 'M1', 'M2', 'M3', 'M4', 'M6', 'M7', 'M8',
       'M9', 'V1', 'V3', 'V4', 'V6', 'V8', 'V11', 'V13', 'V14', 'V17',
       'V20', 'V23', 'V26', 'V27', 'V30', 'V36', 'V37', 'V40', 'V41',
       'V44', 'V47', 'V48', 'V54', 'V56', 'V59', 'V62', 'V65', 'V67',
       'V68', 'V70', 'V76', 'V78', 'V80', 'V82', 'V86', 'V88', 'V89',
       'V91', 'V107', 'V108', 'V111', 'V115', 'V117', 'V120', 'V121',
       'V123', 'V124', 'V127', 'V129', 'V130', 'V136', 'V138', 'V139',
       'V142', 'V147', 'V156', 'V160', 'V162', 'V165', 'V166', 'V169',
       'V171', 'V173', 'V175', 'V176', 'V178', 'V180', 'V182', 'V185',
       'V187', 'V188', 'V198', 'V203', 'V205', 'V207', 'V209', 'V210',
       '

Local validation

In [72]:
idxT = X_train.index[:3*len(X_train)//4]
idxV = X_train.index[3*len(X_train)//4:]

In [77]:
import xgboost as xgb
print("XGBoost version:", xgb.__version__)

XGBoost version: 1.0.1


In [81]:
clf = xgb.XGBClassifier( 
        n_estimators=2000,
        max_depth=12, 
        learning_rate=0.02, 
        subsample=0.8,
        colsample_bytree=0.4, 
        missing=-1, 
        eval_metric='auc',
        # USE CPU
        nthread=4,
        tree_method='hist' 
        # USE GPU
        # tree_method='gpu_hist' 
    )

h = clf.fit(X_train.loc[idxT,cols], 
            y_train[idxT], 
            eval_set=[(X_train.loc[idxV,cols],y_train[idxV])],
            verbose=50, early_stopping_rounds=100)

[0]	validation_0-auc:0.80424
Will train until validation_0-auc hasn't improved in 100 rounds.
[50]	validation_0-auc:0.87514
[100]	validation_0-auc:0.89150
[150]	validation_0-auc:0.90720


KeyboardInterrupt: 